## **Fine-tuning BERT for Bangla NER**

In this notebook, we are going to use **BertForTokenClassification** which is included in the [Transformers library](https://github.com/huggingface/transformers) by HuggingFace. This model has BERT as its base architecture, with a token classification head on top, allowing it to make predictions at the token level, rather than the sequence level. Named entity recognition is typically treated as a token classification problem, so that's what we are going to use it for.

BreakDown the Task list:

1. Environment setup
2. Data processing
3. Data Analysis
4. Load Pre-train model
5. Data Normilization for training
6. Training
7. Inference


#### **Importing Python Libraries and preparing the environment**

This notebook assumes that you have the following libraries installed:
* pandas
* numpy
* sklearn
* pytorch
* transformers
* seqeval

As we are running this in Google Colab, the only libraries we need to additionally install are transformers and seqeval (GPU version):

In [1]:
# !pip install transformers seqeval[gpu] pandas wget

# Donwload Dataset


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# cp /content/drive/MyDrive/bangla_ner_data.zip ./

In [4]:
# !unzip bangla_ner_data.zip

In [5]:
# # Dataset Link
# import wget
# wget.download("https://raw.githubusercontent.com/banglakit/bengali-ner-data/master/main.jsonl")

In [6]:
# !pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

/home/sayan/anaconda3/envs/bert_ner/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


As deep learning can be accellerated a lot using a GPU instead of a CPU, make sure you can run this notebook in a GPU runtime (which Google Colab provides for free! - check "Runtime" - "Change runtime type" - and set the hardware accelerator to "GPU").

We can set the default device to GPU using the following code (if it prints "cuda", it means the GPU has been recognized):

In [8]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


#### **Downloading and preprocessing the data**
Named entity recognition (NER) uses a specific annotation scheme, which is defined (at least for European languages) at the *word* level. An annotation scheme that is widely used is called **[IOB-tagging](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)**, which stands for Inside-Outside-Beginning. Each tag indicates whether the corresponding word is *inside*, *outside* or at the *beginning* of a specific named entity. The reason this is used is because named entities usually comprise more than 1 word.

Let's have a look at an example. If you have a sentence like "Barack Obama was born in Hawaï", then the corresponding tags would be   [B-PERS, I-PERS, O, O, O, B-GEO]. B-PERS means that the word "Barack" is the beginning of a person, I-PERS means that the word "Obama" is inside a person, "O" means that the word "was" is outside a named entity, and so on. So one typically has as many tags as there are words in a sentence.

So if you want to train a deep learning model for NER, it requires that you have your data in this IOB format (or similar formats such as [BILOU](https://stackoverflow.com/questions/17116446/what-do-the-bilou-tags-mean-in-named-entity-recognition)). There exist many annotation tools which let you create these kind of annotations automatically (such as Spacy's [Prodigy](https://prodi.gy/), [Tagtog](https://docs.tagtog.net/) or [Doccano](https://github.com/doccano/doccano)). You can also use Spacy's [biluo_tags_from_offsets](https://spacy.io/api/goldparse#biluo_tags_from_offsets) function to convert annotations at the character level to IOB format.

Here, we will use a NER dataset from [Kaggle](https://www.kaggle.com/namanj27/ner-dataset) that is already in IOB format. One has to go to this web page, download the dataset, unzip it, and upload the csv file to this notebook. Let's print out the first few rows of this csv file:

In [9]:
train_file = "./data/bangla_ner_data/train.json"
val_file = "./data/bangla_ner_data/val.json"

In [10]:
import json
def read_json_file(file_path):
  with open(file_path, 'r') as f:
    data = json.load(f)
  return data

# def read_data_file(input_file):
#   with open(input_file, 'r') as f:
#       lines = f.read().split("\n")
#       data = [json.loads(line) for line in lines if line]
#   return data


In [11]:
train_data = read_json_file(train_file)

In [12]:
val_data = read_json_file(val_file)

In [13]:
train_data

[{'id': 12097,
  'paragraphs': [{'sentences': [{'tokens': [{'orth': ':',
        'tag': '-',
        'ner': 'O'},
       {'orth': 'আল', 'tag': '-', 'ner': 'B-PER'},
       {'orth': 'স্কিনিয়ার', 'tag': '-', 'ner': 'L-PER'},
       {'orth': 'গিটার,', 'tag': '-', 'ner': 'O'},
       {'orth': 'পিয়ানো,', 'tag': '-', 'ner': 'O'},
       {'orth': 'ভোকাল,', 'tag': '-', 'ner': 'O'},
       {'orth': 'মগ', 'tag': '-', 'ner': 'O'},
       {'orth': 'সিনথেসাইজার', 'tag': '-', 'ner': 'O'}]}]}]},
 {'id': 6707,
  'paragraphs': [{'sentences': [{'tokens': [{'orth': 'সে',
        'tag': '-',
        'ner': 'O'},
       {'orth': 'একটি', 'tag': '-', 'ner': 'O'},
       {'orth': 'পাওয়ার', 'tag': '-', 'ner': 'O'},
       {'orth': 'জন্য', 'tag': '-', 'ner': 'O'},
       {'orth': 'এতটাই', 'tag': '-', 'ner': 'O'},
       {'orth': 'মরিয়া', 'tag': '-', 'ner': 'O'},
       {'orth': 'যে', 'tag': '-', 'ner': 'O'},
       {'orth': 'সে', 'tag': '-', 'ner': 'O'},
       {'orth': 'মিথ্যাভাবে', 'tag': '-', 'ner': 'O'}

Let's have a look at the different NER tags, and their frequency:

In [14]:
def get_tag_with_frequency(data):
  tags, total_word = {}, 0
  data_list = []
  for d in data:
    # print(d)
    tokens = d["paragraphs"][0]["sentences"][0]["tokens"]
    text_list  = [i["orth"] for i in tokens]
    label = [i["ner"] for i in tokens]
    text_str = " ".join(text_list)
    # print("label", label)
    if "B-PER" in label or "I-PER" in label or "L-PER" in label or "U-PER" in label:
      data_list.append([text_str, text_list, label])
      for token in tokens:
        word, tag = token["orth"], token["ner"]
        if tag not in tags:
          tags[tag] = 1
        else:
          tags[tag] += 1
        total_word+=1
  print("Number of Total word : ", total_word)
  # print(tags)
  return tags, data_list




In [15]:
tags_fre_train, train_data1 = get_tag_with_frequency(train_data)
tags_fre_train

Number of Total word :  74208


{'O': 62994, 'B-PER': 4417, 'L-PER': 4411, 'I-PER': 1290, 'U-PER': 1096}

In [16]:
len(train_data1)

4491

In [17]:
train_data[0]

{'id': 12097,
 'paragraphs': [{'sentences': [{'tokens': [{'orth': ':',
       'tag': '-',
       'ner': 'O'},
      {'orth': 'আল', 'tag': '-', 'ner': 'B-PER'},
      {'orth': 'স্কিনিয়ার', 'tag': '-', 'ner': 'L-PER'},
      {'orth': 'গিটার,', 'tag': '-', 'ner': 'O'},
      {'orth': 'পিয়ানো,', 'tag': '-', 'ner': 'O'},
      {'orth': 'ভোকাল,', 'tag': '-', 'ner': 'O'},
      {'orth': 'মগ', 'tag': '-', 'ner': 'O'},
      {'orth': 'সিনথেসাইজার', 'tag': '-', 'ner': 'O'}]}]}]}

In [18]:
tags_fre_val, val_data1 = get_tag_with_frequency(val_data)
tags_fre_val

Number of Total word :  19169


{'B-PER': 1188, 'I-PER': 327, 'L-PER': 1186, 'O': 16202, 'U-PER': 266}

In [19]:
def merge_dict(train_tags, val_tags):
    merge_data= {**train_tags, **val_tags}
    return merge_data

In [20]:
merge_tags = merge_dict(tags_fre_train, tags_fre_val)

In [21]:
print(merge_tags)

{'O': 16202, 'B-PER': 1188, 'L-PER': 1186, 'I-PER': 327, 'U-PER': 266}


In [22]:
unique_labels = list(merge_tags.keys())

Let's check how many sentences and words (and corresponding tags) there are in this dataset:

In [23]:
# print(train_data[0])

import pandas as pd

train_df = pd.DataFrame(train_data1, columns =['sentence', 'words', 'word_labels'])
val_df = pd.DataFrame(val_data1, columns =['sentence', 'words', 'word_labels'])

In [24]:
train_df.head()

,sentence,words,word_labels
0,": আল স্কিনিয়ার গিটার, পিয়ানো, ভোকাল, মগ সিনথ...","[:, আল, স্কিনিয়ার, গিটার,, পিয়ানো,, ভোকাল,, ...","[O, B-PER, L-PER, O, O, O, O, O]"
1,সে একটি পাওয়ার জন্য এতটাই মরিয়া যে সে মিথ্যা...,"[সে, একটি, পাওয়ার, জন্য, এতটাই, মরিয়া, যে, স...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,পরিস্থিতি শান্ত হলে ঘটনাস্থলে আবদুল জব্বারের ম...,"[পরিস্থিতি, শান্ত, হলে, ঘটনাস্থলে, আবদুল, জব্ব...","[O, O, O, O, B-PER, L-PER, O, O, O, O, O, O]"
3,২০০৭ সালের জুনে ডাইনাস্টি বিজনেস হাউসের কে এম ...,"[২০০৭, সালের, জুনে, ডাইনাস্টি, বিজনেস, হাউসের,...","[O, O, O, O, O, O, B-PER, I-PER, L-PER, O, O, ..."
4,তেজগাঁও থানার উপপরিদর্শক ( এসআই ) সাইফুল ইসলাম...,"[তেজগাঁও, থানার, উপপরিদর্শক, (, এসআই, ), সাইফু...","[O, O, O, O, O, O, B-PER, L-PER, O, O, O, O, O..."


Let's check how many sentences and words (and corresponding tags) there are in this dataset:

In [25]:
# data.count()

As we can see, there are approximately 48,000 sentences in the dataset, comprising more than 1 million words and tags (quite huge!). This corresponds to approximately 20 words per sentence.

Let's have a look at the different NER tags, and their frequency:

There are 8 category tags, each with a "beginning" and "inside" variant, and the "outside" tag. It is not really clear what these tags mean - "geo" probably stands for geographical entity, "gpe" for geopolitical entity, and so on. They do not seem to correspond with what the publisher says on Kaggle. Some tags seem to be underrepresented. Let's print them by frequency (highest to lowest):

Let's remove "art", "eve" and "nat" named entities, as performance on them will probably be not comparable to the other named entities.

We create 2 dictionaries: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook.

In [26]:
# labels_to_ids = {k: v for v, k in enumerate(data.Tag.unique())}
# ids_to_labels = {v: k for v, k in enumerate(data.Tag.unique())}
# labels_to_ids

In [27]:
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}
labels_to_ids

{'O': 0, 'B-PER': 1, 'L-PER': 2, 'I-PER': 3, 'U-PER': 4}

As we can see, there are now only 10 different NER tags.

Now, we have to ask ourself the question: what is a training example in the case of NER, which is provided in a single forward pass? A training example is typically a **sentence**, with corresponding IOB tags. Let's group the words and corresponding tags by sentence:

In [28]:
# # let's create a new column called "sentence" which groups the words by sentence
# data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# # let's also create a new column called "word_labels" which groups the tags by sentence
# data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
# data.head()

Let's only keep the "sentence" and "word_labels" columns, and drop duplicates:

In [29]:
# data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
# data.head()
val_df

,sentence,words,word_labels
0,র্যান্ডাল এল শোয়ার্টজ একটি মুখবন্ধ এবং প্রযুক...,"[র্যান্ডাল, এল, শোয়ার্টজ, একটি, মুখবন্ধ, এবং,...","[B-PER, I-PER, L-PER, O, O, O, O, O, O, O]"
1,"চৌধূরী , আসমা আব্বাসী , গাজী মাজহারুল আনোয়ার ...","[চৌধূরী, ,, আসমা, আব্বাসী, ,, গাজী, মাজহারুল, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,অপর গোলটি ড্যানিয়েল অ্যাগারের ।,"[অপর, গোলটি, ড্যানিয়েল, অ্যাগারের, ।]","[O, O, B-PER, L-PER, O]"
3,"ভেনিজুয়েলা, রেজি ওটেরো দ্বারা চালিত, ২ ৪ রেকর...","[ভেনিজুয়েলা,, রেজি, ওটেরো, দ্বারা, চালিত,, ২,...","[O, B-PER, L-PER, O, O, O, O, O, O, O, O, O]"
4,প্রধান গায়ক অ্যালেক্স গ্রিনওয়াল্ড বলেছেন যে ...,"[প্রধান, গায়ক, অ্যালেক্স, গ্রিনওয়াল্ড, বলেছে...","[O, O, B-PER, L-PER, O, O, O, O, O, O, O, O, O..."
...,...,...,...
1148,১৮১৮ সালে জ্যাকব হাবনার এর নামকরণ করা হয়েছিল।,"[১৮১৮, সালে, জ্যাকব, হাবনার, এর, নামকরণ, করা, ...","[O, O, B-PER, L-PER, O, O, O, O]"
1149,এইভাবে তিনি রজার ফেদেরার ছাড়াও একমাত্র পুরুষ ...,"[এইভাবে, তিনি, রজার, ফেদেরার, ছাড়াও, একমাত্র,...","[O, O, B-PER, L-PER, O, O, O, O, O, O, O, O, O..."
1150,"গেডি লি - বেস গিটার, সিনথেসাইজার, ভোকাল","[গেডি, লি, -, বেস, গিটার,, সিনথেসাইজার,, ভোকাল]","[B-PER, L-PER, O, O, O, O, O]"
1151,"মাইক গডউইন , তারপর ফাউন্ডেশনের আইনি কাউন্সিল, ...","[মাইক, গডউইন, ,, তারপর, ফাউন্ডেশনের, আইনি, কাউ...","[B-PER, L-PER, O, O, O, O, O, O, O, O, O, O, O..."


In [30]:
val_df.head()

,sentence,words,word_labels
0,র্যান্ডাল এল শোয়ার্টজ একটি মুখবন্ধ এবং প্রযুক...,"[র্যান্ডাল, এল, শোয়ার্টজ, একটি, মুখবন্ধ, এবং,...","[B-PER, I-PER, L-PER, O, O, O, O, O, O, O]"
1,"চৌধূরী , আসমা আব্বাসী , গাজী মাজহারুল আনোয়ার ...","[চৌধূরী, ,, আসমা, আব্বাসী, ,, গাজী, মাজহারুল, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,অপর গোলটি ড্যানিয়েল অ্যাগারের ।,"[অপর, গোলটি, ড্যানিয়েল, অ্যাগারের, ।]","[O, O, B-PER, L-PER, O]"
3,"ভেনিজুয়েলা, রেজি ওটেরো দ্বারা চালিত, ২ ৪ রেকর...","[ভেনিজুয়েলা,, রেজি, ওটেরো, দ্বারা, চালিত,, ২,...","[O, B-PER, L-PER, O, O, O, O, O, O, O, O, O]"
4,প্রধান গায়ক অ্যালেক্স গ্রিনওয়াল্ড বলেছেন যে ...,"[প্রধান, গায়ক, অ্যালেক্স, গ্রিনওয়াল্ড, বলেছে...","[O, O, B-PER, L-PER, O, O, O, O, O, O, O, O, O..."


Let's verify that a random sentence and its corresponding tags are correct:

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [31]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 20
VALID_BATCH_SIZE = 10
EPOCHS = 40
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('csebuetnlp/banglabert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


A tricky part of NER with BERT is that BERT relies on **wordpiece tokenization**, rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level!

For example, if you have word like "Washington" which is labeled as "b-gpe", but it gets tokenized to "Wash", "##ing", "##ton", then one approach could be to handle this by only train the model on the tag labels for the first word piece token of a word (i.e. only label "Wash" with "b-gpe"). This is what was done in the original BERT paper, see Github discussion [here](https://github.com/huggingface/transformers/issues/64#issuecomment-443703063).

Note that this is a **design decision**. You could also decide to propagate the original label of the word to all of its word pieces and let the model train on this. In that case, the model should be able to produce the correct labels for each individual wordpiece. This was done in [this NER tutorial with BERT](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L118). Another design decision could be to give the first wordpiece of each word the original word label, and then use the label “X” for all subsequent subwords of that word. All of them seem to lead to good performance.

Below, we define a regular PyTorch [dataset class](https://pytorch.org/docs/stable/data.html) (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above. Word pieces that should be ignored have a label of -100 (which is the default `ignore_index` of PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)).

For more information about BERT's inputs, see [here](https://huggingface.co/transformers/glossary.html).








In [32]:
labels_to_ids

{'O': 0, 'B-PER': 1, 'L-PER': 2, 'I-PER': 3, 'U-PER': 4}

In [33]:
x = [1,2]
cls = ["U-PER"]
unique_cls = ["B-PER", "I-PER", "L-PER", "U-PER"]

doc_product = len(x)-2

# print(doc_product)
# if doc_product==2:
#   word_label_ids =  ["B-PER", "L-PER"]
# else:
#   word_label_ids =  ["B-PER"]+["I-PER"]*doc_product + ["L-PER"]


# word_label_ids

doc_product = len(x) - 2
print(doc_product)

word_label_ids = ["B-PER", "L-PER"] if doc_product == 2 else ["B-PER"] + ["I-PER"] * doc_product + ["L-PER"]
print(word_label_ids)


0
['B-PER', 'L-PER']


In [34]:



def data_processor(
    text,
    labels,
    tokenizer,
    class_to_index,
    selected_class,
    max_seq_len,
    device,
    inference_mode=False,
    ):

    if inference_mode and not labels:
        labels = ["O"] * len(text)  # to do
    # print(" text lenght : ", len(text))
    # print("label lenght : ", len(labels))

    assert len(text) == len(labels)


    input_ids, label_ids = [], []

    idx = 0
    token_to_word = [None]

    for word, label in zip(text, labels):

        word_tokens_ids = tokenizer.encode(word, add_special_tokens=False)

        # print("word            : ", word)
        # print("word_tokens_ids : ", word_tokens_ids)
        # print("label           : ", label)

        if len(input_ids) + len(word_tokens_ids) - 2 > max_seq_len:
            break

        cls_name = label.split("-")[-1].strip()


        # Before
        # if cls_name in selected_class:
            # word_label_ids = [class_to_index[f"B-{cls_name}"]] + [
            #     class_to_index[f"I-{cls_name}"]
            # ] * (len(word_tokens_ids) - 1)

        # TODO by Saiful

        if label in selected_class:
          if len(word_tokens_ids) > 1:
            if label=="B-PER":
              word_label_ids = [class_to_index[label]] + [
                  class_to_index[f"I-{cls_name}"]
                  ]* (len(word_tokens_ids)-1)
            elif label=="I-PER":
              word_label_ids = [class_to_index[label]]*len(word_tokens_ids)
            elif label=="L-PER":
              word_label_ids = [class_to_index[f"I-{cls_name}"]]* (len(word_tokens_ids))
            else:
              if len(word_tokens_ids)-2 == 0:
                word_label_ids = [class_to_index["B-PER"]]+[class_to_index["I-PER"]]
              else:
                word_label_ids = [class_to_index["B-PER"]]+[class_to_index["I-PER"]]*(len(word_tokens_ids)-1)
          else:
            word_label_ids = [class_to_index["B-PER"]]

        else:
            word_label_ids = [class_to_index["O"] for i in range(len(word_tokens_ids))]

        # print("word label id : ", word_label_ids)

        label_ids.extend(word_label_ids)

        input_ids.extend(word_tokens_ids)

        token_to_word.extend([idx] * len(word_label_ids))
        idx += 1

    # print("input_ids", len(input_ids), input_ids)
    # print("label_ids", len(label_ids), label_ids)
    if len(input_ids) + len(input_ids) - 2 > max_seq_len:
      input_ids = input_ids[:max_seq_len-2]
      label_ids = label_ids[:max_seq_len-2]

    assert len(input_ids) == len(label_ids)

    attention_mask_len = len(input_ids) + 2
    input_ids = (
        [tokenizer.cls_token_id]
        + input_ids
        + [tokenizer.pad_token_id]
        + [tokenizer.pad_token_id] * (max_seq_len - len(input_ids) - 2)
    )
    label_ids = [-100] + label_ids + [-100] * (max_seq_len - len(label_ids) - 1)

    attention_mask = torch.zeros(len(label_ids))
    attention_mask[:attention_mask_len] = 1

    preocessed_data = {
        "input_ids": torch.LongTensor(input_ids).to(device),
        "label_ids": torch.LongTensor(label_ids).to(device),
        "attention_mask": torch.Tensor(attention_mask).to(device),
    }

    if inference_mode:
        for k, v in preocessed_data.items():
            preocessed_data[k] = v.unsqueeze(0)

        preocessed_data["words"] = text
        preocessed_data["word_index"] = token_to_word
    preocessed_data["attention_mask_len"] = attention_mask_len

    return preocessed_data

In [35]:

# data = [
#     ': আল স্কিনিয়ার গিটার, পিয়ানো, ভোকাল, মগ সিনথেসাইজার',
#     [
#       ':',
#       'আল',
#       'স্কিনিয়ার',
#       'গিটার,',
#       'পিয়ানো,',
#       'ভোকাল,',
#       'মগ',
#       'সিনথেসাইজার'
#     ],
#    ['O', 'B-PER', 'L-PER', 'O', 'O', 'O', 'O', 'O']
#   ]

class_to_index =  labels_to_ids
selected_class = ["B-PER", "I-PER", "L-PER", "U-PER"]
# for i in train_data:
#   text = i[1]
#   labels = i[2]
max_seq_len = 512
#   data_processor(
#       text,
#       labels,
#       tokenizer,
#       class_to_index,
#       selected_class,
#       max_seq_len,
#       device,
#   )


In [36]:

DEBUG_CUSTOM_TOKEN_PROCESSING = False

class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):


        item= data_processor(
            self.data.words[index],
            self.data.word_labels[index],
            self.tokenizer,
            class_to_index,
            selected_class,
            self.max_len,
            device
          )

        return item

  def __len__(self):
        return self.len

In [37]:
# data = pd.DataFrame(processed_data, columns=['sentence', 'words', 'word_labels'])

In [38]:
# data.head()

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split:

In [39]:
# train_size = 0.8
# train_dataset = data.sample(frac=train_size, random_state=200)
# test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
# train_dataset = train_dataset.reset_index(drop=True)

# print("FULL Dataset: {}".format(data.shape))
# print("TRAIN Dataset: {}".format(train_dataset.shape))
# print("TEST Dataset: {}".format(test_dataset.shape))



In [40]:
training_set = dataset(train_df, tokenizer, max_seq_len)

In [41]:
testing_set = dataset(val_df, tokenizer, max_seq_len)

In [42]:
training_set[0]

{'input_ids': tensor([    2,    30,  1108,  1450,  5876, 15441,    16, 17691,  3351,    16,
          3323,  1173,    16,  8446,  3397,   960,  4864,  1595,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

Let's have a look at the first training example:

In [43]:
training_set[0]

{'input_ids': tensor([    2,    30,  1108,  1450,  5876, 15441,    16, 17691,  3351,    16,
          3323,  1173,    16,  8446,  3397,   960,  4864,  1595,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

Let's verify that the input ids and corresponding targets are correct:

In [44]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["label_ids"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
:           0
আল          1
স্ক         3
##িনিয়ার   3
গিটার       0
,           0
পিয়া       0
##নো        0
,           0
ভো          0
##কাল       0
,           0
মগ          0
সিন         0
##থে        0
##সাই       0
##জার       0
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -1

Now, let's define the corresponding PyTorch dataloaders:

In [45]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Note that only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights, which we will train, together with the pretrained weights, using our labelled dataset. This is also printed as a warning when you run the code cell below.

Then, we move the model to the GPU.

In [46]:
model = BertForTokenClassification.from_pretrained('csebuetnlp/banglabert', num_labels=len(labels_to_ids))
model.to(device)

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['encoder.layer.4.output.LayerNorm.weight', 'encoder.layer.3.attention.self.key.bias', 'encoder.layer.2.attention.self.key.weight', 'encoder.layer.4.attention.output.dense.bias', 'encoder.layer.3.attention.output.dense.bias', 'encoder.layer.7.attention.self.key.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.1.attention.output.LayerNorm.weight', 'encoder.layer.9.attention.output.dense.bias', 'encoder.layer.11.attention.self.value.weight', 'encoder.layer.5.attention.self.key.bias', 'encoder.layer.7.attention.self.value.weight', 'encoder.layer.3.attention.output.dense.weight', 'encoder.layer.11.attention.output.LayerNorm.bias', 'encoder.layer.2.intermediate.dense.bias', 'encoder

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

#### **Training the model**

Before training the model, let's perform a sanity check, which I learned thanks to Andrej Karpathy's wonderful [cs231n course](http://cs231n.stanford.edu/) at Stanford (see also his [blog post about debugging neural networks](http://karpathy.github.io/2019/04/25/recipe/)). The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83.

Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/17. The loss for a given token will thus be -ln(1/17). As PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (which is used by `BertForTokenClassification`) uses *mean reduction* by default, it will compute the mean loss for each of the tokens in the sequence for which a label is provided.

Let's verify this:



In [47]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["label_ids"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(1.6241, device='cuda:0', grad_fn=<NllLossBackward0>)

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [48]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 5])

Next, we define the optimizer. Here, we are just going to use Adam with a default learning rate. One can also decide to use more advanced ones such as AdamW (Adam with weight decay fix), which is [included](https://huggingface.co/transformers/main_classes/optimizer_schedules.html) in the Transformers repository, and a learning rate scheduler, but we are not going to do that here.

In [49]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on [a really good repository about multilingual NER](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L344).

In [50]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['label_ids'].to(device, dtype = torch.long)

        # print()

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels).to_tuple()

        # print("loss : ", type(loss), loss)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps

    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [51]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['label_ids'].to(device, dtype = torch.long)

            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels).to_tuple()

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

And let's train the model!

In [52]:
EPOCHS = 50

import os
directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)
    valid(model, testing_loader)

    tokenizer.save_vocabulary(directory)
        # save the model weights and its configuration file
    model.save_pretrained(directory)

Training epoch: 1
Training loss per 100 training steps: 1.8300338983535767
Training loss per 100 training steps: 0.6112256088469288
Training loss per 100 training steps: 0.5666795149074858
Training loss epoch: 0.5603312019507091
Training accuracy epoch: 0.8085598383750289
Validation loss per 100 evaluation steps: 0.5231549143791199
Validation loss per 100 evaluation steps: 0.5078902008509872
Validation Loss: 0.5130866493644386
Validation Accuracy: 0.816138657843309
Training epoch: 2
Training loss per 100 training steps: 0.4387921392917633
Training loss per 100 training steps: 0.45345566325848646
Training loss per 100 training steps: 0.43567462480483365
Training loss epoch: 0.43085870610343086
Training accuracy epoch: 0.8415563616134568
Validation loss per 100 evaluation steps: 0.3870715796947479
Validation loss per 100 evaluation steps: 0.4004630326929659
Validation Loss: 0.40681502004635745
Validation Accuracy: 0.8498075314940966
Training epoch: 3
Training loss per 100 training steps:

#### **Evaluating the model**

Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits.

In [142]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['label_ids'].to(device, dtype = torch.long)

            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels).to_tuple()

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 0.9705618473452632.

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.4829927980899811
Validation loss per 100 evaluation steps: 0.3293700642626742
Validation loss per 100 evaluation steps: 0.331556095490904
Validation loss per 100 evaluation steps: 0.3156554575794762
Validation loss per 100 evaluation steps: 0.31130318397779033
Validation loss per 100 evaluation steps: 0.3161905466638096
Validation Loss: 0.31313948812218123
Validation Accuracy: 0.9237239440644682


In [ ]:
type(labels)

list

In [ ]:
type(predictions)

list

However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library:

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

         PER       0.56      0.61      0.59      2307

   micro avg       0.56      0.61      0.59      2307
   macro avg       0.56      0.61      0.59      2307
weighted avg       0.56      0.61      0.59      2307



Performance already seems quite good, but note that we've only trained for 1 epoch. An optimal approach would be to perform evaluation on a validation set while training to improve generalization.

#### **Inference**

The fun part is when we can quickly test the model on new, unseen sentences.
Here, we use the prediction of the **first word piece of every word** (which is how the model was trained).

*In other words, the code below does not take into account when predictions of different word pieces that belong to the same word do not match.*

In [56]:
# sentence = "India has a capital called Mumbai. On wednesday, the president will give a presentation"
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

tokenizer = BertTokenizerFast.from_pretrained('./model')
model = BertForTokenClassification.from_pretrained('./model', num_labels=len(labels_to_ids))
model.to(device)

text_list = [
    ": আল স্কিনিয়ার গিটার, পিয়ানো, ভোকাল, মগ সিনথেসাইজার",
    "আব্দুর রহিম নামের কাস্টমারকে একশ টাকা বাকি দিলাম",
    "রহিম নামের কাস্টমারকে একশ টাকা বাকি দিলাম",
    "সাইফুল ইসলাম ন্যাচারাল ল্যাঙ্গুয়েজে প্রসেসিং খুব বেশি ভালো পারে না । তাই সে বেশি বেশি স্টাডি করতেছে।",
    "ডিপিডিসির স্পেশাল টাস্কফোর্সের প্রধান মুনীর চৌধুরী জানান",
    "তিনি মোহাম্মদ বাকির আল-সদর এর ছাত্র ছিলেন।",
    "লিশ ট্র্যাক তৈরির সময় বেশ কয়েকজন শিল্পীর দ্বারা অনুপ্রাণিত হওয়ার কথা স্মরণ করেন, বিশেষ করে ফ্রাঙ্ক সিনাত্রা ।",
]

sentence = text_list[2]

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
    continue
  else:
    word_level_predictions.append(pair[1])

# we join tokens, if they are not special ones
print(wp_preds)
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

[('[CLS]', 'O'), ('রহিম', 'B-PER'), ('নামের', 'O'), ('কাস', 'I-PER'), ('##ট', 'I-PER'), ('##মার', 'I-PER'), ('##কে', 'I-PER'), ('একশ', 'O'), ('টাকা', 'O'), ('বাকি', 'O'), ('দিলাম', 'O'), ('[SEP]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'

Performance already seems quite good, but note that we've only trained for 1 epoch. An optimal approach would be to perform evaluation on a validation set while training to improve generalization.

#### **Inference**

The fun part is when we can quickly test the model on new, unseen sentences.
Here, we use the prediction of the **first word piece of every word** (which is how the model was trained).

*In other words, the code below does not take into account when predictions of different word pieces that belong to the same word do not match.*

#### **Saving the model for future use**

Finally, let's save the vocabulary (.txt) file, model weights (.bin) and the model's configuration (.json) to a directory, so that both the tokenizer and model can be re-loaded using the `from_pretrained()` class method.


In [ ]:
import os

directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


## Legacy

The following code blocks were used during the development of this notebook, but are not included anymore.

In [ ]:
def prepare_sentence(sentence, tokenizer, maxlen):
      # step 1: tokenize the sentence
      tokenized_sentence = tokenizer.tokenize(sentence)

      # step 2: add special tokens
      tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"]

      # step 3: truncating/padding
      if (len(tokenized_sentence) > maxlen):
        # truncate
        tokenized_sentence = tokenized_sentence[:maxlen]
      else:
        # pad
        tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]

      # step 4: obtain the attention mask
      attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

      # step 5: convert tokens to input ids
      ids = tokenizer.convert_tokens_to_ids(tokenized_sentence)

      return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(attn_mask, dtype=torch.long),
            #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
      }

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [labels_to_ids[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
sentence = "this is a test @huggingface".strip().split()

inputs = tokenizer(
    sentence,
    return_offsets_mapping=True,
    padding='max_length',
    truncation=True
    )
print(inputs)
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
token_offsets = inputs["offset_mapping"]
print(tokens)
print(token_offsets)

{'input_ids': [[101, 2023, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
word = "@huggingface"

inputs = tokenizer(word, return_offsets_mapping=True, padding='max_length', truncation=True)
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
token_offsets = inputs["offset_mapping"]
print(tokens)
print(token_offsets)

['[CLS]', '@', 'hugging', '##face', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '